The scope of ETL project is as follows:
We have NFL player data from Wikipedia , Kaggle and Data.World and would want information on players who made it Pro-Bowl from 2017 to 1997
1. So we get the players name(list) from Wikipedia for each year's probowl selection ( Check pages from 2017--1997)
2. Once we get the information for 30 years of data , we remove the duplicates
3. then we join the player name with information ( csv file and spreadsheet from Kaggle and Data.World) to get additional information on the player.
4. We clean the data
5. We save the data to a cloud database and expose it as REST API's..


In [34]:
# Import BeautifulSoup
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import bamboolib as bam

In [35]:
# Import Splinter and set the chromedriver path
from splinter import Browser
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

In [36]:
base_url = 'https://en.wikipedia.org/wiki/'
player_info = []

In [37]:
# Visit the following URL
# Our scope is All-Pro-Team from 1997 to 2017
nflseason_year = 1997
while nflseason_year < 2018:
    url = base_url + str(nflseason_year)+'_All-Pro_Team'
    browser.visit(url)
    time.sleep(3)
    #getting the html content from the browser instance launched using splinter
    html=browser.html
    #scrape the web page using beautiful soup and lxml parser
    soup = BeautifulSoup(html, 'lxml')
    player_tableColl = soup.find_all('table', class_='wikitable')
    for table in player_tableColl:
        tr_coll = table.find_all('tr')
        for item in tr_coll:
            data = item.find_all(['th','td'])
            if data:
                try:
                    player = data[1].a.text
                    player1 = data[2].a.text
                    if player not in player_info:
                        player_info.append(player)
                    if player1 not in player_info:
                        player_info.append(player1)
                except Exception:pass
                print("exception")
    nflseason_year = nflseason_year + 1
    
#print the dictionary
print(player_info)
    

exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
